# Stiffness Analysis

I want to make a comprehensive analysis of stiffness and comparison of all the methods I've implemented. I should also include the Adams-Moulden or whatever.

Some potentially useful resources:

- Section 4.6 of the book
- https://en.wikipedia.org/wiki/Stiff_equation
- https://www.mathworks.com/company/technical-articles/stiff-differential-equations.html
- https://web.archive.org/web/20230926233120/http://acmbulletin.fiit.stuba.sk/vol4num3/satek.pdf
- https://math.stackexchange.com/questions/4217585/stiff-odes-trouble-detecting-stiffness-from-the-plot-of-an-ode?noredirect=1&lq=1

### Basic imports and idk

In [6]:
import numpy as np
import matplotlib.pyplot as plt

### Implementation of main algorithms

There is something wrong with something here, I don't quite know which algorithm has an incorrect implementation. Maybe RKF45.

I should standardize the solvers, rn they use different variables with different names in different ways.

In [1]:
def euler_method(F, t_span, x0, n_steps):
    dim = len(x0)
    t_start, t_end = t_span[0], t_span[1]
    h = (t_end - t_start) / (n_steps - 1)

    t = np.linspace(t_start, t_end, n_steps)
    x = np.zeros((n_steps, dim))
    x[0] = x0

    for n in range(1, n_steps):
        x[n] = x[n-1] + h * F(t[n-1], x[n-1])

    return t, x

In [2]:
def RK_method(F, t_span, x0, n_steps, a, b, c):
    dim = len(x0)
    l = len(b)
    t_start, t_end = t_span
    t_step = (t_end - t_start) / (n_steps - 1)

    t = np.linspace(t_start, t_end, n_steps)
    x = np.zeros((n_steps, dim))
    x[0] = x0
    k = np.zeros((l, dim))

    for n in range(1, n_steps):
        k[0] = F(t[n-1], x[n-1])
        for i in range(1, l):
            t_i = t[n-1] + t_step * c[i]
            x_i = x[n-1] + t_step * np.dot(a[i-1][:i], k[:i])
            k[i] = F(t_i, x_i)
        x[n] = x[n-1] + t_step * np.dot(b, k)

    return t, x

In [3]:
# 3-stage Runge-Kutta
def RK3(F, t_span, x0, n_steps):
    a = np.array(
        [[0.5, 0], [-1, 2]]
    )
    b = np.array([1/6, 4/6, 1/6])
    c = np.array([0, 0.5, 1])
    return RK_method(F, t_span, x0, n_steps, a, b, c)

# Classical (4-stage) Runge-Kutta
def RK4(F, t_span, x0, n_steps):
    a = np.array(
        [[0.5, 0, 0],
         [0, 0.5, 0],
         [0, 0, 1]]
    )
    b = np.array([1/6, 2/6, 2/6, 1/6])
    c = np.array([0, 0.5, 0.5, 1])
    return RK_method(F, t_span, x0, n_steps, a, b, c)

I could improve the Adam's-Bashforth solver, maybe it's not well coded, I could also add the implicit version

In [4]:
def compute_a_adams(k):
    # return ndarray of a[i]'s
    # which are the integral of pochhammer over i!, 0 <= i < k
    # the book gives a recursive formula to compute a[i]
    a = np.ones((k))
    for i in range(1, k):
        for j in range(i):
            a[i] -= a[j] / (i+1-j)
    return a

def compute_delF(k, F, t, x):
    # t = [t_n-k, ..., t_n-1]
    # x = [x_n-k, ..., x_n-1]
    dim = len(x[0])
    delF = np.zeros((k, k, dim))
    for j in range(k):
        delF[0, j, :] = F(t[j], x[j])
    for i in range(1, k):
        for j in range(i, k):
            delF[i, j] = delF[i-1, j] - delF[i-1, j-1]
    return delF[:, k-1]

def adams_bashforth(F, t_span, x0, n_steps, k):
    dim = len(x0)
    t_start, t_end = t_span
    h = (t_end - t_start) / (n_steps - 1)

    t = np.linspace(t_start, t_end, n_steps)
    x = np.zeros((n_steps, dim))
    a = compute_a_adams(k)

    # Compute first k steps with RK
    A = np.array(
    [[0.5, 0], [-1, 2]]
    )
    B = np.array([1/6, 4/6, 1/6])
    C = np.array([0, 0.5, 1])
    _, x_first_k = RK_method(F, (t_start, t_start + h*(k-1)), x0, k, A, B, C)
    x[:k] = x_first_k

    for n in range(k, n_steps):
        delF = compute_delF(k, F, t[n-k:n], x[n-k:n])
        x[n] = x[n-1] + h * np.dot(a, delF)
    
    return t, x

Also add the generalized Adams-Moulton method.

In [ ]:
def newtons_method(x1, f, df, n):
    # Initial guess
    x = x1
    for i in range(n):
        if f(x) != 0 and df(x) != 0:
            x -= f(x) / df(x)
        else:
            return x
    return x

def implicit_euler(F, dF_dx, t_span, x0, n_steps, k):
    t_start, t_end = t_span
    h = (t_end - t_start) / (n_steps - 1)

    t = np.linspace(t_start, t_end, n_steps)
    x = np.zeros(n_steps)
    x[0] = x0
    
    for n in range(1, n_steps):
        f = lambda y: x[n-1] + h * F(t[n], y) - y
        df = lambda y: h * dF_dx(t[n], y) - 1
        x[n] = newtons_method(x[n-1], f, df, k)
    
    return t, x